## Motion Correction using the NoRMCorre algorithm

In [1]:
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import matplotlib.pyplot as plt
import numpy as np
import os



import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params


try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl
bpl.output_notebook()


from fcutils.file_io.utils import get_file_name


Loading BokehJS ...

In [2]:
# Get files to process
fld = "D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05"
fnames    = [os.path.join(fld, '19JUN05_BF164p1_v1_ds126_crop_ffcSub.tif')]  # ffc filename to be processed
fname_raw = [os.path.join(fld, '19JUN05_BF164p1_v1_ds126_crop_raw.tif')]  ## raw file

# Filenames for saved data
fname = get_file_name(fnames[0])
pw_rigid_savename = os.path.join(fld, fname + "-pwrigidmc.mmap")
rigid_savename = os.path.join(fld, fname + "-rigidmc.mmap")


## Set up cluster

In [3]:
# start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

## Set up MC params

In [4]:
# dataset dependent parameters
frate = 10.                       # movie frame rate
decay_time = 2.              # length of a typical transient in seconds

#static parameters
#pw_rigid = True  # flag for performing rigid or piecewise rigid motion correction
border_nan = 'copy'  # replicate values along the boundary (if True, fill in with NaN)

# motion correction parameters
mc_dict = {
    'gSig_filt': (2,2), # used for high pass filtering in 1p data
    'strides': (30,30), # Can make it shakey
    'overlaps': (11,11), #
    'max_shifts': (25,25), # max shifts in rigid. makes a difference. usually 5-8. can be larger if motion is very rigid
    'max_deviation_rigid': 7, # non-rigid shift deviation from rigid max shifts. makes a difference
    'fnames': fnames, # ! using the ffc subtracted data
    'fr': frate,
    'decay_time': decay_time,
    'border_nan': border_nan,
    'shifts_opencv': True,
    'num_frames_split':100,
    'nonneg_movie': True,


}

opts = params.CNMFParams(params_dict=mc_dict)


# Motion Correction
The background signal in micro-endoscopic data is very strong and makes the motion correction challenging. 
As a first step the algorithm performs a high pass spatial filtering with a Gaussian kernel to remove the bulk of the background and enhance spatial landmarks. 
The size of the kernel is given from the parameter `gSig_filt`. If this is left to the default value of `None` then no spatial filtering is performed (default option, used in 2p data).
After spatial filtering, the NoRMCorre algorithm is used to determine the motion in each frame. The inferred motion is then applied to the *original* data so no information is lost.

The motion corrected files are saved in memory mapped format. If no motion correction is being performed, then the file gets directly memory mapped.

### Rigid motion correction first

In [5]:
mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))

mc.fname_tot_rig = rigid_savename, # filename for output of rigid motion correction
mc.fname_tot_els = pw_rigid_savename, # filename for output of piece wise rigid motion correction



In [6]:
# correct for rigid motion correction and save the file (in memory mapped form)
mc.motion_correct(save_movie=True)

print(f"\n\nMotion corrected video was saved at: {mc.mmap_file}")

Motion corrected video was saved at: ['D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05\\19JUN05_BF164p1_v1_ds126_crop_ffcSub_rig__d1_109_d2_92_d3_1_order_F_frames_22662_.mmap']


### Piecewise rigid motion correction

In [11]:
# motion correct piecewise rigid
mc.pw_rigid = True  # turn the flag to True for pw-rigid motion correction
mc.template = mc.mmap_file  # use the results of the rigid motion corrction to save in computation (optional)
mc.motion_correct(save_movie=True, template=mc.total_template_rig)



mes: [19862 19863 19864 19865 19866 19867 19868 19869 19870 19871 19872 19873
 19874 19875 19876 19877 19878 19879 19880 19881 19882 19883 19884 19885
 19886 19887 19888 19889 19890 19891 19892 19893 19894 19895 19896 19897
 19898 19899 19900 19901 19902 19903 19904 19905 19906 19907 19908 19909
 19910 19911 19912 19913 19914 19915 19916 19917 19918 19919 19920 19921
 19922 19923 19924 19925 19926 19927 19928 19929 19930 19931 19932 19933
 19934 19935 19936 19937 19938 19939 19940 19941 19942 19943 19944 19945
 19946 19947 19948 19949 19950 19951 19952 19953 19954 19955 19956 19957
 19958 19959 19960 19961]
      783353 [motion_correction.py:motion_correction_piecewise():3023] [19980] Processing: frames: [19962 19963 19964 19965 19966 19967 19968 19969 19970 19971 19972 19973
 19974 19975 19976 19977 19978 19979 19980 19981 19982 19983 19984 19985
 19986 19987 19988 19989 19990 19991 19992 19993 19994 19995 19996 19997
 19998 19999 20000 20001 20002 20003 20004 20005 20006 20007 20008 

In [7]:
### Visualize results

In [14]:
# load motion corrected movie
m_rig = cm.load(mc.mmap_file)

# visualize templates
plt.figure(figsize = (20,10))
plt.imshow(mc.total_template_rig, cmap = 'gray')


0%|          | 0/1 [00:00<?, ?it/s]      939535 [movies.py:                load():1708] [19980] loading mmap file in memory
      941126 [movies.py:                load():1711] [19980] mmap
100%|██████████| 1/1 [00:01<00:00,  1.67s/it]


# Stop cluster

In [1]:
cm.stop_server(dview=dview)

NameError: name 'cm' is not defined